In [1]:
from bs4 import BeautifulSoup as bs
from bs4 import Tag, NavigableString
import re
import requests as r
import pandas as pd
import numpy as np
import datetime
import csv
import os
import sys
from state_cleaner import *
from selenium import webdriver
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
os.chdir('../..')
os.chdir('CovidMobile')
os.getcwd()

'/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile'

In [3]:
def check_save(file_path):
    path = ('data/{}'.format(file_path))
    if os.path.exists(path):
        os.remove(path)
    else:
        print("{} does not exist".format(file_path))
    

def write_out(file_path, url):
    path = ('data/{}'.format(file_path))
    print(file_path)
    with open(path, mode='w', newline='') as file:
        file_writer = csv.writer(file, delimiter=',')
        print(url)
        with r.get(url, verify=False, stream=True) as res:
            lines = (line.decode('utf-8',errors='ignore').replace('\0','') for line in res.iter_lines())
            for row in csv.reader(lines):
                print(row)
                file_writer.writerow(row)

                
                
def create_stuff(file_path,url):
    check_save(file_path)
    write_out(file_path,url)

### Google Mobility Data

In [29]:
#Downloads the .csv file of all mobility data from the google mobility site
goog_files = os.getcwd()+'/data'
my_url = 'https://www.google.com/covid19/mobility/'

html= r.get(my_url)
soup = bs(html.content,'lxml')
current_link = ''
for link in soup.find_all('a'):
    current_link = link.get('href')
    if current_link.endswith('csv'):
        print(current_link)
        myfile = r.get(current_link)
        open('{}/{}'.format(goog_files, current_link.rsplit('/', 1)[1]),'wb').write(myfile.content)

https://www.gstatic.com/covid19/mobility/Global_Mobility_Report.csv


### ARDA CSV Download

In [5]:
#arda_files = os.getcwd()+'/data'
my_url = 'http://www.thearda.com/Archive/Files/Downloads/RCMSMGCY_DL2.asp'

html= r.get(my_url)
soup = bs(html.content,'lxml')
current_link = ''
for link in soup.find_all('a'):
    current_link = link.get('href')
    if '2658v' in current_link:
        xlsx_file = r.get(current_link)
        output = open('ARDA_Congregation_Data.csv', 'wb')
        output.write(xlsx_file.content)
        output.close()
        
#move file into data folder:
path = os.getcwd()
os.rename(path+'/ARDA_Congregation_Data.csv', path+'/data/ARDA_Congregation_Data.csv')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\Noah\\CovidMobile/ARDA_Congregation_Data.csv' -> 'C:\\Users\\Noah\\CovidMobile/data/ARDA_Congregation_Data.csv'

### Closures and SAH Order Dates:

from: https://www.kff.org/report-section/state-data-and-policy-actions-to-address-coronavirus-sources/

In [3]:
url = 'https://www.kff.org/report-section/state-data-and-policy-actions-to-address-coronavirus-sources/'
html= r.get(url)
soup = bs(html.content,'lxml')

states = [str(x.text).strip('<b>/') for x in soup.findAll('b')]
#closure_dates = pd.DataFrame(states, columns = ['States'])

text = [str(x.text).strip('\n\t<b>/') for x in soup.findAll(['b', 'span'])]
text = text[10:]

In [43]:
#do the rest in a loop
for i, state in enumerate(states):
    this_state = text.index(state)
    next_state = text.index(states[i])
    for line in text[this_state+1:next_state]:
        if line[0].isdigit() and re.search('[A-Z]', line):
            d = line[0:re.search('[A-Z]', line).start()-1]
            w = line[re.search('[A-Z]', line).start():]
            new_line = pd.Series({'Date': d, 'Description': w, 'State': state})
            closure_dates = closure_dates.append(new_line,ignore_index=True)
            
this_state = text.index('WYOMING')
for line in text[this_state+1:len(text)-1]:
    if line[0].isdigit() and re.search('[A-Z]', line):
        d = line[0:re.search('[A-Z]', line).start()-1]
        w = line[re.search('[A-Z]', line).start():]
        new_line = pd.Series({'Date': d, 'Description': w, 'State': 'WYOMING'})
        closure_dates = closure_dates.append(new_line,ignore_index=True)
        
closure_dates.to_csv('/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/data/closure_dates.csv')

In [44]:
closure_dates

,State,Date,Description
0,ALABAMA,3/13,Public Health Emergency:
1,ALABAMA,"3/17, 3/19, 3/20, 3/26, 3/27","School Closures, Large Gatherings Ban, Bar/Res..."
2,ALABAMA,3/18,Primary Election Postponement:
3,ALABAMA,3/23,Section 1135 Waiver:
4,ALABAMA,"3/27, 4/3","Non-Essential Business Closures, Stay At Home ..."
...,...,...,...
2714,WYOMING,3/13,"Emergency Declaration, Public Health Emergency:"
2715,WYOMING,"3/19, 3/27, 4/3","School Closures, Bar/Restaurant Limits:"
2716,WYOMING,"3/20, 3/27, 4/3",Large Gatherings Ban:
2717,WYOMING,3/27,Section 1135 Waiver:


### Download Harvard Voting Data

In [14]:
from selenium import webdriver

url= 'https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/VOQCHQ'
driver = webdriver.Safari()
driver.get(url)


download = driver.find_element_by_xpath('//*[@id="datasetForm:tabView:filesTable:1:j_idt1227"]')
driver.execute_script("arguments[0].click()", download)
driver.quit()

os.rename('/Users/samismalling/Downloads/countypres_2000-2016.csv', '/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/data/countypres_2000-2016.csv')

### Food Environment Atlas, USDA Data

In [16]:
#Downloads into .xls file because the data comes in multiple sheets 
usda_files = os.getcwd()+'/data'
my_url = 'https://www.ers.usda.gov/data-products/food-environment-atlas/data-access-and-documentation-downloads/'

html= r.get(my_url)
soup = bs(html.content,'lxml')
current_link = ''
for link in soup.find_all('a'):
    current_link = link.get('href')
    if 'DataDownload' in current_link:
        dwnld_link = 'https://www.ers.usda.gov'+current_link
        print(dwnld_link)
        myfile = r.get(dwnld_link)
        open('{}/{}'.format(usda_files, 'food_env_atlas_usda.xls'),'wb').write(myfile.content)

https://www.ers.usda.gov/webdocs/DataFiles/48731/DataDownload.xls?v=0


### Apple Data

In [21]:
status = 0
attempt = 0
while status != 200:
    appl_day = str(datetime.datetime.now().date() - datetime.timedelta(attempt))
    url = "https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-{}.csv".format(appl_day)
    print(url)
    status = r.get(url, verify=False, stream=True).status_code
    attempt = attempt + 1
    print(status,attempt)

https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-21.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


404 1
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-20.csv
404 2
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-19.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


404 3
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-18.csv
404 4
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-17.csv


...

C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


404 6
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-15.csv
404 7
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-14.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


404 8
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-13.csv
200 9


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


In [30]:
apple_file = 'apple_data_{}.csv'.format(appl_day)
create_stuff(apple_file, url)

apple_data_2020-04-13.csv does not exist
apple_data_2020-04-13.csv
https://covid19-static.cdn-apple.com/covid19-mobility-data/2005HotfixDev13/v1/en-us/applemobilitytrends-2020-04-13.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'covid19-static.cdn-apple.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


## Census Data


In [8]:
start = 'https://web.archive.org/web/20150807220054/http://quickfacts.census.gov/qfd/download_data.html'

In [9]:
res = r.get(start, verify=False)
soup = bs(res.content,'html.parser')
sys.path.insert(0, "/notebooks")

options = webdriver.ChromeOptions()

driver = webdriver.Chrome()
driver.get(start)



for link in soup.findAll('a', href=True):
    if '.txt' in link['href']:
        url = link['href']
        print(link)
        driver.find_element_by_xpath('//a[@href="'+link['href']+'"]').click()
        url = driver.find_element_by_id("playback").get_attribute("src")
        create_stuff('{}_{}'.format(link['href'].rsplit('/')[-1],str(datetime.datetime.now().date())),url)
        driver.execute_script("window.history.go(-1)")


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a href="download/DataSet.txt">DataSet.txt</a>
DataSet.txt_2020-04-21 does not exist
DataSet.txt_2020-04-21
https://web.archive.org/web/20150821182814if_/http://quickfacts.census.gov:80/qfd/download/DataSet.txt


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a href="download/DataDict.txt">DataDict.txt</a>
DataDict.txt_2020-04-21 does not exist
DataDict.txt_2020-04-21
https://web.archive.org/web/20150821061818if_/http://quickfacts.census.gov:80/qfd/download/DataDict.txt


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


...

C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a href="download/FootNotes.txt">FootNotes.txt</a>
FootNotes.txt_2020-04-21 does not exist
FootNotes.txt_2020-04-21
https://web.archive.org/web/20150905124937if_/http://quickfacts.census.gov/qfd/download/FootNotes.txt


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


<a href="download/FootText.txt">FootText.txt</a>
FootText.txt_2020-04-21 does not exist
FootText.txt_2020-04-21
https://web.archive.org/web/20150717163825if_/http://quickfacts.census.gov:80/qfd/download/FootText.txt


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'web.archive.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


# USDA data

In [5]:
usda = 'https://www.ers.usda.gov/'
location = 'data-products/county-level-data-sets/download-data/'
res = r.get(usda+location)
soup = bs(res.content,'html.parser')
for hr in soup.findAll('a',href=True):
    if ".xls" in hr['href']:
        print(hr['href'].rsplit('/')[-1].split('.')[0])
        df = pd.read_excel(usda+hr['href'])
        head = np.where(df.T.nunique()==max(df.T.nunique()))[0][0]+1
        df = pd.read_excel(usda+hr['href'],header=head)
        df.to_csv('data/{}.csv'.format(hr['href'].rsplit('/')[-1].split('.')[0]),index=False)

Education
Unemployment
PovertyEstimates
PopulationEstimates


# ZIP Codes to County File

using crosswalk from this source: https://www.huduser.gov/portal/datasets/usps_crosswalk.html

In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

url= 'https://www.kaggle.com/danofer/zipcodes-county-fips-crosswalk'
driver = webdriver.Safari()
driver.get(url)
driver.find_element_by_link_text("Download (2 MB)").click()
#log into Kaggle
driver.find_element_by_link_text("Sign in with your email")
username = driver.find_element_by_id("textfield-qolce1ya1w")
username.clear()
username.send_keys("ss2676@cornell.edu")

password = driver.find_element_by_name("pwd")
password.clear()
password.send_keys("catskillz")
#download = driver.find_element_by_xpath('//a[@href="/danofer/zipcodes-county-fips-crosswalk/download"]')
#driver.execute_script("arguments[0].click()", download)
driver.quit()


In [30]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select

url= 'https://www.huduser.gov/portal/datasets/usps_crosswalk.html#data'
driver = webdriver.Safari()
driver.get(url)

select1 = Select(driver.find_element_by_id("crosswalk"))
select1.select_by_visible_text("ZIP-COUNTY")

select2 = Select(driver.find_element_by_id("year"))
select2.select_by_visible_text("1st Quarter 2020")

#time.sleep(5)

download = driver.find_element_by_xpath('//input[@onclick="downloadFile()"]')
driver.execute_script("arguments[0].click()", download)
driver.quit()

#os.rename('/Users/samismalling/Downloads/countypres_2000-2016.csv', '/Users/samismalling/Documents/mobility-report-data-extractor-master/CovidMobile/data/countypres_2000-2016.csv')

In [31]:
print(download)

<selenium.webdriver.remote.webelement.WebElement (session="4B5EFD26-7657-4022-A3ED-07D02FEA4132", element="node-58E4F2FF-32CE-4C1D-8476-35D2A675A590")>


# New York Times - Covid Cases

In [5]:
nyt = 'https://github.com/nytimes/covid-19-data'
res = r.get(nyt)
res

<Response [200]>

In [8]:
soup = bs(res.content,'html.parser')
for hr in soup.findAll('a',href=True):
    if '/nytimes/covid-19-data/master/' in hr['href']:
        print(hr['href'])
        create_stuff('/{}_{}'.format(str(datetime.datetime.now().date()),hr['href'].rsplit('/')[-1]),hr['href'])
        

https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv
/2020-04-21_us.csv does not exist
/2020-04-21_us.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv
/2020-04-21_us-states.csv does not exist
/2020-04-21_us-states.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv
/2020-04-21_us-counties.csv does not exist
/2020-04-21_us-counties.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv
/2020-04-21_us.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv
/2020-04-21_us-states.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv
/2020-04-21_us-counties.csv
https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv


C:\Users\Noah\Anaconda3\lib\site-packages\urllib3\connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'raw.githubusercontent.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


# Neighboring county map

In [11]:
url = 'https://www2.census.gov/geo/docs/reference/county_adjacency.txt'
df = pd.read_csv(url,delimiter='\t',encoding='ISO-8859-1',header=None)
df = df.ffill()
df.columns = ['county','FIPS','nearby_county','nearby_FIPS'] 
df.to_csv('data/county_adjacency.csv',index=False)

# wikipedia dates for emergency declarations. 


In [12]:
Wiki_t = "https://en.wikipedia.org/wiki/U.S._state_and_local_government_response_to_the_2020_coronavirus_pandemic"

In [13]:
def grab_col_headers(tup_list, x, y,heads):
    for element in tup_list:
        if element[0] == x:
            for el in y:
                heads.extend([el[1]])
            continue
        else:
            heads.extend([element[1]])
    return heads

In [14]:
res = r.get(Wiki_t)

In [15]:

soup = bs(res.content,'lxml')
cols = []
index = []
tmp = soup.find('table', class_='wikitable').findAll('tr')

first = tmp[0]
    
header = []
heads=[]
if first.find('th').has_attr("rowspan"):
    rowspn = int(first.find('th')['rowspan'])
    first = tmp[0:rowspn]
    allRows = tmp[rowspn:-1]
    for th_no, data in enumerate(first):
        for td_no, row in enumerate(data.findAll('th')):
            print(row)
            if th_no == 0:
                if row.has_attr("colspan"):
                    header.append(((int(row['colspan'])), row.text))
                else:
                    header.append((0, row.text))
            else:
                cols.append((0,row.text))
    header = grab_col_headers(header,max([n[0] for n in header]),cols,heads)            
            
    print(header)       
else:
    first = tmp[0]
    header = [header.get_text() for header in first.find_all('th')]
    allRows = tmp[1:-1]

results = []
for row in allRows:
    vals =[]
    for v in row:       
        if isinstance(v, NavigableString):
            continue
        if isinstance(v, Tag):
            vals.extend([v.text])
    results.append(vals)


df = pd.DataFrame(data=results, columns=header)


<th colspan="2" rowspan="2">State/territory
</th>
<th rowspan="2">State of emergency declared
</th>
<th rowspan="2"><a href="/wiki/Stay-at-home_order" title="Stay-at-home order">Stay at home ordered</a>
</th>
<th rowspan="2">Gatherings banned
</th>
<th rowspan="2">Out-of-state travel restrictions
</th>
<th colspan="4">Closures ordered
</th>
<th rowspan="2">Sources
</th>
<th>Schools</th>
<th>Daycares</th>
<th>Bars &amp; sit-down restaurants</th>
<th>Non-essential retail
</th>
['State/territory\n', 'State of emergency declared\n', 'Stay at home ordered\n', 'Gatherings banned\n', 'Out-of-state travel restrictions\n', 'Schools', 'Daycares', 'Bars & sit-down restaurants', 'Non-essential retail\n', 'Sources\n']


In [16]:
df.replace(regex={r'\n': ''},inplace=True)
df.columns = pd.Series(df.columns.values).str.replace('\n','')
df.to_csv('data/emergency_orders.csv')

# Updated Census Data

In [9]:
df_county_names = pd.read_csv('data/FIPS_CountyName.txt_2020-04-16', sep='delimiter', header=None)
df_county_names = df_county_names[0].str.split(' ',n=1,expand=True)
df_county_names = pd.concat([df_county_names[0],df_county_names[1].str.split(',',n=1,expand=True)],axis=1)
df_county_names.columns = ['FIPS','County','State']
df_county_names['State'] = df_county_names['State'].str.replace(' ', '')
df_county_names.shape,df_county_names.head()
df_county_names['state'] = df_county_names['State'].apply(replace_acronym)

C:\Users\Noah\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [ ]:
df_county_names

In [ ]:
df_county_names['strng'] = df_county_names['County'].str.replace(' ','').str.lower() + df_county_names['state'].str.replace(' ','').str.lower()

In [ ]:
key_file = open("census_api.txt")
lines = key_file.readlines()
census_api_key = lines[0].rstrip()

In [22]:
import pandas as pd

#allows us to store results of API call cleanly
import json

'''#get list of all zipcodes in Los Angeles County separated by commas
laZips = open('laZips.txt', 'r').readlines()
laZips = [z.replace('\n', '') for z in laZips]
laZips = ','.join(laZips)'''

#put your census API key here
apiKey = census_api_key

#construct the API call we will use
url = 'https://api.census.gov/data/2017/acs/acs1?get=NAME,B02015_009E,B02015_009M&for=county:*&key={}'.format(census_api_key)
#call the API and collect the response
response = r.get(url)

In [8]:
import pandas as pd

#allows us to store results of API call cleanly
import json

'''#get list of all zipcodes in Los Angeles County separated by commas
laZips = open('laZips.txt', 'r').readlines()
laZips = [z.replace('\n', '') for z in laZips]
laZips = ','.join(laZips)'''

#put your census API key here
apiKey = census_api_key

#construct the API call we will use
url = 'https://api.census.gov/data/2017/acs/acs1?get=NAME,B02015_009E,B02015_009M&for=county:*&key={}'.format(census_api_key)
#call the API and collect the response
response = requests.get(url)

#load the response into a JSON, ignoring the first element which is just field labels
formattedResponse = json.loads(response.text)[1:]

#flip the order of the response from [population, zipcode] -> [zipcode, population]
formattedResponse = [item[::-1] for item in formattedResponse]

#store the response in a dataframe
laZipPopulations = pd.DataFrame(columns=['zipcode', 'population'], data=formattedResponse)

#save that dataframe to a CSV spreadsheet
laZipPopulations.to_csv('laZipPopulations.csv', index=False)

3143

In [9]:
for i in range(1,len(url)+1):
    try:
        census = 'https://www.census.gov/quickfacts/fact/table/' + ','.join(url.head(300).values) + ',US/PST045219'
        r.get(census)
        print(i)
    except:
        break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178


In [17]:
census = 'https://www.census.gov/quickfacts/fact/table/' + ','.join(url.tail(10).values) + ',US/PST045219'
tst = r.get(census)

In [18]:
census

'https://www.census.gov/quickfacts/fact/table/niobraracountywyoming,parkcountywyoming,plattecountywyoming,sheridancountywyoming,sublettecountywyoming,sweetwatercountywyoming,tetoncountywyoming,uintacountywyoming,washakiecountywyoming,westoncountywyoming,US/PST045219'

In [ ]:
https://www.census.gov/quickfacts/fact/table/washingtoncountycolorado,washingtoncountyalabama,US/PST045219